In [ ]:
from flask import Flask, request, jsonify
import joblib
import pandas as pd
from flask_cors import CORS
import random

app = Flask(__name__)
CORS(app)

# Load the trained model and label encoders
model = joblib.load('category_model.pkl')
le_category1 = joblib.load('le_category1.pkl')
le_category2 = joblib.load('le_category2.pkl')
le_category3 = joblib.load('le_category3.pkl')

# Load the original dataset to fetch recommendations
data = pd.read_csv("D:/feelings.csv")
print(data.head())  # Log the first few rows to check the data

@app.route('/predict', methods=['POST'])
def predict():
    data_input = request.json
    physical_feeling = data_input.get('physical_feeling')
    emotional_feeling = data_input.get('emotional_feeling')
    mental_feeling = data_input.get('mental_feeling')

    print(f"Inputs: {physical_feeling}, {emotional_feeling}, {mental_feeling}")  # Log the inputs

    try:
        # Encode the inputs
        physical_encoded = le_category1.transform([physical_feeling])[0]
        emotional_encoded = le_category2.transform([emotional_feeling])[0]
        mental_encoded = le_category3.transform([mental_feeling])[0]

        # Make prediction
        prediction = model.predict([[physical_encoded, emotional_encoded, mental_encoded]])

        # Filter recommendations based on categories
        recommendations = data[
            (data['category1'] == physical_feeling) &
            (data['category2'] == emotional_feeling) &
            (data['category3'] == mental_feeling)
        ]['recommendation'].tolist()

        # Randomly select one recommendation
        if recommendations:
            random_recommendation = random.choice(recommendations)
            return jsonify({'recommendation': random_recommendation})
        else:
            return jsonify({'recommendation': 'No recommendations found for these feelings.'})
    except Exception as e:
        print(f"Error: {str(e)}")  # Log the error message
        return jsonify({'error': str(e)}), 400

if __name__ == '__main__':
    app.run()


   id category1 category2 category3                          recommendation
0   1      sick     happy      calm            Take a warm shower to relax.
1   2      sick     happy      calm  Read a good book to distract yourself.
2   3      sick     happy      calm              Watch your favorite movie.
3   4      sick     happy      calm        Try some herbal tea for comfort.
4   5      sick     happy      calm                Listen to calming music.
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Oct/2024 17:14:23] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2024 17:14:23] "POST /predict HTTP/1.1" 200 -


Inputs: sick, sad, anxious


127.0.0.1 - - [22/Oct/2024 17:14:51] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2024 17:14:52] "POST /predict HTTP/1.1" 200 -


Inputs: sick, sad, stressed


127.0.0.1 - - [22/Oct/2024 18:15:49] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Oct/2024 18:15:49] "POST /predict HTTP/1.1" 200 -


Inputs: sick, sad, anxious
